VIC Pricing Model - This notebook is for pricing life settlements.

Import and define functions

In [117]:
import pandas as PD
import numpy as NP
# import matplotlib.pyplot as PLT
# import seaborn as SB
# import scipy.stats as ST
# import sklearn.model_selection as MS
# import sklearn.preprocessing as PR
# import sklearn.linear_model as LM

def YEARS_BETWEEN(early_date, later_date):
    return PD.Timestamp(later_date).year - PD.Timestamp(early_date).year - ((PD.Timestamp(later_date).month, PD.Timestamp(later_date).day) < (PD.Timestamp(early_date).month, PD.Timestamp(early_date).day))

def MONTHS_BETWEEN(date1, date2):
    dt1 = PD.Timestamp(date1)
    dt2 = PD.Timestamp(date2)
    return (dt2.year - dt1.year) * 12 + dt2.month - dt1.month - (dt2.day < dt1.day)

def ADD_YEARS(date, years):
    dt = PD.Timestamp(date)
    try:
        return dt.replace(year = dt.year + years)
    except ValueError:
        return dt.replace(month = 2, day = 28, year = dt.year + years)

def AGE_AT(date_of_birth, reference_date):
    return YEARS_BETWEEN(date_of_birth, reference_date)

Initialize pricing inputs - NEED TO MOVE TO JSON INPUT

In [118]:
PRICING_ASSUMPTIONS = PD.read_csv("Data/Pricing_Assumptions.csv")

HURDLE_RATE = PRICING_ASSUMPTIONS.loc[0, "Hurdle_Rate"]                         # annual rate

print(HURDLE_RATE)

# GROSS_PURCHASE_PRICE = 1000000                                                # dollars
UW_SPOT_ADJ = 1.0                                                               # multiplier
UW_WEAR_OFF_DUR = 25                                                            # years

0.5


Initialize policy inputs - NEED TO MOVE TO JSON INPUT

In [119]:
DATE_OF_BIRTH = '1981-06-26'                # YYYY-MM-DD
ISSUED_FACE_AMOUNT = 1000000                # dollars
GENDER = 'Other'                            # 'M', 'F' or 'Other'
ISSUE_DATE = '2024-04-25'                   # YYYY-MM-DD
PRODUCT_TYPE = 'Term'                       # 'Term', 'WL', etc.
PRODUCT_DURATION = 20                       # years
PREMIUM_MODE = 'Annual'                     # 'Annual', 'Monthly', etc.
ISSUE_STATE = 'CA'                          # two-letter state code
MODAL_PREMIUM = 6000                        # dollars
CLOSING_DATE = '2026-06-25'                 # YYYY-MM-DD

Starting calcs

In [120]:
ISSUED_DURATION = YEARS_BETWEEN(ISSUE_DATE, CLOSING_DATE)
print(ISSUED_DURATION)

if PRODUCT_DURATION == "Lifetime": 
  PRODUCT_DURATION_CALC = 121 - AGE_AT(ISSUE_DATE, DATE_OF_BIRTH)
else:
    PRODUCT_DURATION_CALC = PRODUCT_DURATION

MATURITY_DATE = ADD_YEARS(ISSUE_DATE, PRODUCT_DURATION_CALC)
ISSUE_AGE = AGE_AT(DATE_OF_BIRTH, ISSUE_DATE)
MORTALITY_TBL_BASE = "VBT15_" 

if GENDER == "M":
    MORTALITY_TBL_BASE = MORTALITY_TBL_BASE + "MNS_ALB"
else:
    MORTALITY_TBL_BASE = MORTALITY_TBL_BASE + "FNS_ALB"

MORTALITY_TBL_FILENAME = MORTALITY_TBL_BASE + ".csv"

ISSUE_DATE_DAY = PD.Timestamp(ISSUE_DATE).day
CLOSING_DATE_DAY = PD.Timestamp(CLOSING_DATE).day
print(ISSUE_DATE_DAY, CLOSING_DATE_DAY)

2
25 25


Load mortality table

In [121]:
MORTALITY_TBL = PD.read_csv("Data/" + MORTALITY_TBL_FILENAME)
MORTALITY_TBL.set_index('Iss_Age', inplace = True)              # Set index to 'Iss_Age' for proper lookups
# print(MORTALITY_TBL.head())

def MORTALITY_RATE(issue_age, duration):
    if issue_age + duration >= 121:
        return 0
    else:
        return MORTALITY_TBL.loc[issue_age + max(0, duration - 26), str(min(26, duration))]       
    
# mortality_rate_example = print(MORTALITY_RATE(19, 5))
# mortality_rate_example = print(MORTALITY_RATE(19, 25))
# mortality_rate_example = print(MORTALITY_RATE(19, 26))
# mortality_rate_example = print(MORTALITY_RATE(19, 27))

Initialize Pricing Model Data Frame

In [ ]:
# Initialize Pricing Model Data Frame

# Calculate total months for the projection
TOTAL_MONTHS = PRODUCT_DURATION_CALC * 12

# Calculate starting policy year
POLICY_YEAR_START = ISSUED_DURATION + 1

# Generate lists for each column
LS_MONTH = list(range(1, TOTAL_MONTHS + 1))  # 1 to 240
# POLICY_MONTH = LS_MONTH  # Assuming same as life settlement month for now

# Generate dates starting from closing date
CLOSING_DATE_TS = PD.Timestamp(CLOSING_DATE)
FIRST_MONTHIVERSARY = CLOSING_DATE_TS.replace(day = PD.Timestamp(ISSUE_DATE).day) + PD.DateOffset(months = 1 * (CLOSING_DATE_DAY >= ISSUE_DATE_DAY))  # Next monthiversary after closing
# print(FIRST_MONTHIVERSARY)

END_DATES = [FIRST_MONTHIVERSARY + PD.DateOffset(months = i) + PD.DateOffset(days = -1) for i in range(TOTAL_MONTHS)]
# print(END_DATES[:12])  # Print first 12 end dates as a sample

BEG_DATES = [CLOSING_DATE_TS] + [END_DATES[i-1] + PD.DateOffset(days = 1) for i in range(1, TOTAL_MONTHS)]  # First is closing, then next day after each previous end date
# print(BEG_DATES[:12])  # Print first 12 beginning dates as a sample

# Calculate Policy_Duration and Mortality_Rate outside the DataFrame
POLICY_DURATION = [YEARS_BETWEEN(ISSUE_DATE, END_DATES[i]) for i in range(TOTAL_MONTHS)]
MORTALITY_RATES = [MORTALITY_RATE(ISSUE_AGE, POLICY_DURATION[i]) for i in range(TOTAL_MONTHS)]

PRICING_MODEL_DF = PD.DataFrame({
    'LS_Month': LS_MONTH,
    # 'Policy_Month': POLICY_MONTH,
    'Policy_Year': [POLICY_YEAR_START + (m-1)//12 for m in LS_MONTH],
    'Policy_Duration': POLICY_DURATION,
    'Beg_Date': BEG_DATES,
    'End_Date': END_DATES,
    'Days_In_Month':  [ (END_DATES[i] - BEG_DATES[i]).days + 1 for i in range(TOTAL_MONTHS) ],
    'Mortality_Rate': MORTALITY_RATES
})

# Set pandas display options for wider output
PD.set_option('display.width', 1200)
PD.set_option('display.max_columns', None)

print(PRICING_MODEL_DF.head(25))  # Print first 12 rows of the DataFrame


    LS_Month  Policy_Year  Policy_Duration   Beg_Date   End_Date  \
0          1            3                2 2026-06-25 2026-07-24   
1          2            3                2 2026-07-25 2026-08-24   
2          3            3                2 2026-08-25 2026-09-24   
3          4            3                2 2026-09-25 2026-10-24   
4          5            3                2 2026-10-25 2026-11-24   
5          6            3                2 2026-11-25 2026-12-24   
6          7            3                2 2026-12-25 2027-01-24   
7          8            3                2 2027-01-25 2027-02-24   
8          9            3                2 2027-02-25 2027-03-24   
9         10            3                2 2027-03-25 2027-04-24   
10        11            3                3 2027-04-25 2027-05-24   
11        12            3                3 2027-05-25 2027-06-24   
12        13            4                3 2027-06-25 2027-07-24   
13        14            4                3 2027-